# «SC2 min20» Report

Report for the "simple_call_2" (simple call) dataset with config "min20" (all included labels have at least 20 samples).
This test includes lstm, cnn 1d and 2d, each with image and hog features.

 - label extracting config: exact at human labeled starts and ends. Additionally right padded with silent audio parts to same length
 - hog config: ??
 - spectrogram config: sampling_rate=500000, x_pixels_per_sec=2000, window='Ham'

K-Fold with K=8 for 310+91+124+153+132+467 samples, 8 bins of 11+11+11+11+11+11 and a remainder of 222+3+36+65+44+379, for each fold: 1079 training, 66 validation, and 132 testing samples
 - val-bins: 1
 - test-bins: 2

In [ ]:
import glob
from IPython.display import display
from engine.settings import BSC_ROOT_DATA_FOLDER
from matplotlib import pyplot as plt
from utils.report import create_sorted_result

bin_size = 11
labels = ['B2', 'B3', 'B4', 'VS', 'VSV', 'UPS']
sorted_results = create_sorted_result('testing_sc2_min20')

## Results of the audio transforms (spectrograms)

20 randomly taken samples per label

In [1]:
from utils.report import create_spectrogram_overview
image_paths = glob.glob(str(BSC_ROOT_DATA_FOLDER / 'simple_call_2' / 'audio' / 'sr500000_xpps2000_h512_wHam') + '/*.png')
create_spectrogram_overview(image_paths, labels, bin_size)

## Overview about the models

Kfold results over every model type

In [ ]:
from utils.report import create_model_overview
create_model_overview(sorted_results)

## Statistics

- Error-plots train + valid
- Confusion matrix per report per result

In [ ]:
from utils.report import create_statistics
create_statistics(sorted_results)

## identifies important pixels

Layer-Wise Relevance Propagation with bounded deep taylor

In [ ]:
import os
from utils.report import nn_cnn_2d_lrp

for result in sorted_results:
    recognizeBestAction = list(filter(lambda action: action['task'] == 'ParallelTask' and 'action' in action['props'] and action['props']['action'] == 'recognize_best_val_acc', result['pipeline']))
    model_path = recognizeBestAction[0]['props']['kwargs']['model_path']
    fig = nn_cnn_2d_lrp(os.path.dirname(model_path) + '/epoch_100.h5.json', 4)
    display(fig)
    fig.clear()
    plt.close(fig)
    break